# Production Runbook


In [ ]:
import sys, subprocess
from pathlib import Path

print("Python:", sys.executable)
repo_root = Path.cwd().parent if Path.cwd().name == "notebooks" else Path.cwd()
subprocess.run(["pip", "install", "-e", str(repo_root)], check=True)


In [ ]:
from pathlib import Path
import shutil

raw_csv = repo_root / "data" / "raw" / "time_series_60min_singleindex.csv"
if not raw_csv.exists():
    nested = next((repo_root / "data" / "raw").glob("opsd-time_series-*/time_series_60min_singleindex.csv"), None)
    if nested:
        shutil.copy(nested, raw_csv)
        print("Copied:", nested, "->", raw_csv)
    else:
        print("Missing OPSD CSV. Please download or place it in data/raw/")
print("Raw CSV:", raw_csv)


In [ ]:
import subprocess

subprocess.run([
    "python", "-m", "gridpulse.data_pipeline.validate_schema",
    "--in", "data/raw", "--report", "reports/data_quality_report.md"
], check=True, cwd=str(repo_root))

subprocess.run([
    "python", "-m", "gridpulse.data_pipeline.build_features",
    "--in", "data/raw", "--out", "data/processed"
], check=True, cwd=str(repo_root))

subprocess.run([
    "python", "-m", "gridpulse.data_pipeline.split_time_series",
    "--in", "data/processed/features.parquet",
    "--out", "data/processed/splits"
], check=True, cwd=str(repo_root))


End‑to‑end runbook for production‑style execution.

## 1) Data → Features → Splits

In [ ]:
from pathlib import Path
import shutil

raw_csv = repo_root / "data" / "raw" / "time_series_60min_singleindex.csv"
if not raw_csv.exists():
    nested = next((repo_root / "data" / "raw").glob("opsd-time_series-*/time_series_60min_singleindex.csv"), None)
    if nested:
        shutil.copy(nested, raw_csv)
        print("Copied:", nested, "->", raw_csv)
    else:
        print("Missing OPSD CSV. Please download or place it in data/raw/")
print("Raw CSV:", raw_csv)


## 2) Baselines + GBM

## 3) Deep models (LSTM + TCN)

In [ ]:
import subprocess

subprocess.run(['python', '-m', 'gridpulse.forecasting.train', '--config', 'configs/train_forecast.yaml'], check=True)


## 4) Start API + Dashboard (run in terminals)

- `uvicorn services.api.main:app --reload --port 8000`
- `streamlit run services/dashboard/app.py`


## 5) Monitoring + Optimization

- `curl http://localhost:8000/monitor`
- `curl -X POST http://localhost:8000/optimize -H 'Content-Type: application/json' -d '{"forecast_load_mw":[8000,8200],"forecast_renewables_mw":[3200,3100]}'`


## Visual Sanity Checks

These plots provide a quick visual validation of recent behavior.

In [ ]:
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt

repo_root = Path.cwd().parent if Path.cwd().name == 'notebooks' else Path.cwd()
features_path = repo_root / 'data' / 'processed' / 'features.parquet'
if not features_path.exists():
    print('features.parquet not found. Run the feature pipeline first.')
else:
    df_viz = pd.read_parquet(features_path).sort_values('timestamp')
    if {'load_mw','wind_mw','solar_mw'}.issubset(df_viz.columns):
        recent = df_viz.tail(7 * 24)
        fig, ax = plt.subplots(3, 1, figsize=(12, 7), sharex=True)
        recent.plot(x='timestamp', y='load_mw', ax=ax[0], color='#1f77b4', title='Load (last 7 days)')
        recent.plot(x='timestamp', y='wind_mw', ax=ax[1], color='#2ca02c', title='Wind (last 7 days)')
        recent.plot(x='timestamp', y='solar_mw', ax=ax[2], color='#ff7f0e', title='Solar (last 7 days)')
        plt.tight_layout()
    else:
        print('Expected columns not found in features.parquet')
